Introductie

P1

P2

P3

P4

## De invloed van klimaat en geografie op voedselproductie

Niet elk land beschikt over dezelfde natuurlijke omstandigheden om voedsel te produceren. Factoren zoals neerslag, temperatuur en klimaatzones bepalen welke gewassen efficiënt kunnen groeien. In landen met een ongunstig klimaat moeten gewassen onder intensievere en vaak minder duurzame omstandigheden worden verbouwd.
Daarnaast speelt de geografische ligging een rol in hoe voedsel wordt verhandeld: landen die ver van exportmarkten liggen, hebben hogere transportkosten en extra uitstoot, wat hun voedselsysteem minder duurzaam maakt.
In dit perspectief onderzoeken we hoe deze geografische factoren — natuurlijke omstandigheden en ligging ten opzichte van wereldmarkten — bijdragen aan verschillen in de duurzaamheid van nationale voedselsystemen.

### Klimaatzones en natuurlijke omstandigheden beïnvloeden duurzaamheid
Landen verschillen sterk in klimaat. Tropische regio’s kennen bijvoorbeeld hoge temperaturen en regelmatige regenval, terwijl droge of koude gebieden kampen met beperkte groeiseizoenen en lagere landbouwopbrengsten per hectare.
In warmere, vruchtbare klimaatzones kunnen gewassen natuurlijk groeien, met weinig extra input. In koudere of drogere gebieden zijn irrigatie, verwarming, kunstmest of import van hulpbronnen nodig om dezelfde opbrengst te halen. Deze extra inputs verhogen de uitstoot per kilogram geproduceerd voedsel.
Daarom is het voedselsysteem in sommige landen per definitie minder duurzaam, simpelweg door hun ligging in een ongunstige klimaatzone.

In [56]:
import pandas as pd
import plotly.express as px

# 📁 Laad de datasets
neerslag_df = pd.read_csv("datasets/API_AG.LND.PRCP.MM_DS2_en_csv_v2_4487.csv", skiprows=4)
productie_df = pd.read_csv("datasets/Production_Crops_Livestock_filtered.csv")
emissies_crops_df = pd.read_csv("datasets/Emissions_crops_filtered.csv")
emissies_livestock_df = pd.read_csv("datasets/Emissions_livestock_filtered.csv")

# 🌧️ Verwerk neerslagdata
neerslag_long = neerslag_df.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    var_name="Year",
    value_name="Precipitation_mm"
)
neerslag_long = neerslag_long[["Country Name", "Year", "Precipitation_mm"]]
neerslag_long["Year"] = pd.to_numeric(neerslag_long["Year"], errors="coerce")
neerslag_long = neerslag_long.dropna(subset=["Year", "Precipitation_mm"])
neerslag_long["Country Name"] = neerslag_long["Country Name"].replace({
    "United States": "United States of America"
})

# 🚜 Verwerk productiegegevens
productie_filtered = productie_df[productie_df["Element"] == "Production"]
productie_aggregated = productie_filtered.groupby(["Area", "Year"])["Value"].sum().reset_index()
productie_aggregated.rename(columns={"Area": "Country Name", "Value": "Production_tonnes"}, inplace=True)
productie_aggregated["Year"] = pd.to_numeric(productie_aggregated["Year"], errors="coerce")
productie_aggregated["Country Name"] = productie_aggregated["Country Name"].replace({
    "Netherlands (Kingdom of the)": "Netherlands"
})

# 🌱🐄 Combineer gewas- en veeteeltuitstoot
emissies_crops_total = emissies_crops_df.groupby(["Area", "Year"])["Value"].sum().reset_index()
emissies_livestock_total = emissies_livestock_df.groupby(["Area", "Year"])["Value"].sum().reset_index()

emissies_total = pd.merge(
    emissies_crops_total, emissies_livestock_total,
    on=["Area", "Year"], how="outer", suffixes=("_crops", "_livestock")
)
emissies_total["Emissions_total"] = (
    emissies_total["Value_crops"].fillna(0) + emissies_total["Value_livestock"].fillna(0)
)
emissies_total = emissies_total[["Area", "Year", "Emissions_total"]]
emissies_total.rename(columns={"Area": "Country Name"}, inplace=True)
emissies_total["Year"] = pd.to_numeric(emissies_total["Year"], errors="coerce")

# 🔁 Merge alles samen op land + jaar
emissie_productie_df = pd.merge(emissies_total, productie_aggregated, on=["Country Name", "Year"])
emissie_productie_df = pd.merge(emissie_productie_df, neerslag_long, on=["Country Name", "Year"])

# 🔎 Filter op de 7 landen
landen_selectie = [
    "Nigeria", "South Africa", "India", "Japan",
    "United States of America", "Brazil", "Indonesia"
]
emissie_productie_df = emissie_productie_df[emissie_productie_df["Country Name"].isin(landen_selectie)]

# ➗ Bereken uitstoot per ton
emissie_productie_df["Emissions_per_ton"] = (
    emissie_productie_df["Emissions_total"] / emissie_productie_df["Production_tonnes"]
)

# 📈 Maak scatterplot
fig = px.scatter(
    emissie_productie_df,
    x="Precipitation_mm",
    y="Emissions_per_ton",
    animation_frame="Year",
    color="Country Name",
    text="Country Name",
    hover_name="Country Name",
    title="Uitstoot per ton geproduceerd vs Neerslag (per jaar)",
    labels={
        "Precipitation_mm": "Neerslag (mm/jaar)",
        "Emissions_per_ton": "Uitstoot per ton geproduceerd (Mt/ton)"
    }
)

fig.update_traces(
    textposition='top center',
    marker=dict(size=14)
)

fig.update_layout(
    xaxis=dict(range=[0, 3000]),
    yaxis=dict(range=[0, 6e-7])
)

fig.show()



/var/folders/82/txt4_nkx0l76kq68c7g_klqc0000gn/T/ipykernel_32679/3348794567.py:6: DtypeWarning:

Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.



#### Analyse: Uitstoot per ton geproduceerd vs Neerslag

Deze visualisatie toont hoe efficiënt landen voedsel produceren in termen van uitstoot. Op de x-as staat de jaarlijkse neerslag (in mm), en op de y-as de uitstoot uit landbouw omgerekend naar **uitstoot per geproduceerde ton voedsel** (Mt CO₂e/ton). Elk punt vertegenwoordigt één land in een bepaald jaar. We volgen Brazilië, India, Indonesië, Japan, Nigeria, Zuid-Afrika en 
de Verenigde Staten.

##### Waarom deze landen?

In deze analyse beperken we ons tot zeven landen: Brazilië, India, Indonesië, Japan, Nigeria, Zuid-Afrika en de Verenigde Staten. Deze selectie is bewust gemaakt om drie redenen:

1. **Contrasterende landbouwsystemen**  
   Elk van deze landen vertegenwoordigt een fundamenteel ander landbouwmodel: van grootschalige industriële productie (VS), tot extensieve plantaardige landbouw (India), tot tropisch kleinschalig boerenbedrijf (Nigeria, Indonesië). Door juist deze contrasten te vergelijken, ontstaat inzicht in hoe verschillende strategieën leiden tot uiteenlopende emissiepatronen, los van natuurlijke omstandigheden.

2. **Relevantie voor mondiale voedselvoorziening én uitstoot**  
   De gekozen landen behoren tot de grootste producenten van voedsel in de wereld (India, VS, Brazilië), of zijn door hun bevolkingsomvang of economische groei belangrijk voor de toekomst van landbouwtransitie (zoals Indonesië, Nigeria). Ze dragen significant bij aan zowel de wereldwijde voedselproductie als de landbouwgerelateerde uitstoot, wat hen tot relevante cases maakt.

3. **Beheersbare maar representatieve vergelijking**  
   Door te focussen op zeven landen kunnen we trends in detail analyseren, zonder dat de vergelijking verwatert. Een analyse van alle landen zou leiden tot visuele ruis en afleiding van het doel: het blootleggen van fundamentele verschillen in landbouw-efficiëntie. De gekozen landen bestrijken vijf continenten, meerdere klimaatzones, en uiteenlopende sociaaleconomische structuren, waardoor ze samen een krachtig representatief beeld geven van mondiale landbouwdynamiek.

Kortom: deze landen zijn niet willekeurig gekozen, maar geselecteerd om de kernvraag van deze analyse – **hoe efficiënt produceren landen voedsel in termen van uitstoot?** – zo scherp en inzichtelijk mogelijk te beantwoorden.


##### Wat valt op?

- **De Verenigde Staten heeft structureel een hoge uitstoot per ton geproduceerd.**  
  Hoewel de VS een grote landbouwsector heeft, is de uitstoot per ton voedsel aanzienlijk hoger dan in bijvoorbeeld India of Brazilië. Dit wijst waarschijnlijk op een uitstootintensieve landbouwpraktijk, zoals intensieve veeteelt (runderen, varkens), gebruik van grote hoeveelheden kunstmest, en een hoge mate van mechanisatie en energieverbruik.

- **India en Brazilië combineren hoge productie met relatief lage uitstoot.**  
  India produceert in veel jaren zelfs meer dan de VS, maar doet dit met een aanzienlijk lagere uitstoot per ton. Dit suggereert een meer plantaardige en extensieve landbouw, of efficiënter gebruik van middelen. Ook Brazilië scoort opvallend goed qua uitstoot-efficiëntie, ondanks hoge neerslag.

- **Japan en Zuid-Afrika zitten qua efficiëntie in de middenmoot.**  
  Japan is technologisch geavanceerd maar kleinschalig, wat leidt tot stabiele maar gematigde uitstoot per ton. Zuid-Afrika toont sterke fluctuaties, wat kan wijzen op klimaatinvloeden (zoals droogte), economische instabiliteit of structurele verschillen in landbouwtypes per jaar.

- **Indonesië en Nigeria produceren weinig, en stoten relatief weinig uit.**  
  In beide landen is de landbouw kleinschaliger en minder geïndustrialiseerd. Dat leidt tot lage totale productie, maar ook tot lage uitstoot per ton. Wellicht is er beperkte toegang tot landbouwtechnologie of infrastructuur.

##### Waarom stoten sommige landen meer uit per ton dan andere?

Een hoge uitstoot per ton geproduceerd voedsel kan verschillende oorzaken hebben:
- Dominantie van veeteelt (vooral rundvlees en zuivel -> hoge methaanuitstoot)
- Inefficiënt gebruik van landbouwgrond of input (bijv. overmatig gebruik van kunstmest)
- Mechanisatie die veel fossiele energie verbruikt
- Verspilling in de keten of lage opbrengst per hectare

##### Conclusie

De visualisatie laat overtuigend zien dat **meer neerslag niet automatisch leidt tot efficiëntere of duurzamere landbouw.** India toont aan dat een droog(achtig) klimaat niet hoeft te betekenen dat productie inefficiënt is. Tegelijkertijd laat de VS zien dat landbouwkeuzes een grotere impact hebben dan klimaat: ondanks goede omstandigheden is de uitstoot per ton daar hoog.

Efficiëntie in landbouw wordt dus grotendeels bepaald door **productiestrategie**, niet door geografie. Wie wat produceert – en hóé – bepaalt uiteindelijk de ecologische voetafdruk.



### Internationale handelsstromen beïnvloeden nationale uitstoot

De uitstoot van een nationaal voedselsysteem zegt niet alleen iets over wat er binnen dat land geconsumeerd wordt, maar ook over wat er geproduceerd wordt voor de wereldmarkt. Sommige landen produceren voedsel voornamelijk voor export. In die gevallen wordt de uitstoot lokaal geregistreerd, terwijl de consumptie elders plaatsvindt. 

Omdat deze landen produceren voor internationale markten, ligt hun landbouwuitstoot vaak hoger dan landen met vergelijkbare bevolkingsaantallen of eetgewoonten. Dit betekent dat de uitstoot van een land niet per se iets zegt over het lokale dieet of de binnenlandse voedselvoorziening. Het weerspiegelt eerder hun positie in het mondiale voedselsysteem. 

Dus landbouwuitstoot weerspiegelt niet enkel lokale consumptie, maar wordt sterk beïnvloed door de internationale vraag – landen met hoge export produceren meer en stoten daardoor meer uit, vaak voor andermans bord.


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# === STEP 1: LOAD ALL DATA ===
df_export = pd.read_csv("datasets/filtered_export_data.csv", encoding='latin1')
df_crops = pd.read_csv("datasets/Emissions_crops_filtered.csv")
df_livestock = pd.read_csv("datasets/Emissions_livestock_filtered.csv")
df_pop = pd.read_csv("datasets/Population_E_All_Data_(Normalized).csv")

# === STEP 2: CLEAN EXPORT DATA ===
df_export = df_export[df_export["Element"] == "Export quantity"]
df_export = df_export[df_export["Value"] > 0]
df_export["Item"] = df_export["Item"].str.strip()
df_export = df_export[["Reporter Countries", "Item", "Year", "Value"]]
df_export.columns = ["Country", "Item", "Year", "Tonnes"]

# === STEP 3: CLEAN + MERGE EMISSIONS DATA ===
df_emissions = pd.concat([df_crops, df_livestock])
df_emissions = df_emissions[df_emissions["Value"].notna()]
df_emissions = df_emissions[df_emissions["Value"] > 0]
df_emissions = df_emissions[["Area", "Item", "Year", "Value"]]
df_emissions.columns = ["Country", "Item", "Year", "Emissions_Gg"]
df_emissions["Emissions_kg"] = df_emissions["Emissions_Gg"] * 1e6

# === STEP 4: CALCULATE EMISSION FACTORS (kg CO₂ per tonne) ===
df_merged = pd.merge(df_export, df_emissions, on=["Country", "Item", "Year"], how="left")
df_merged["Emission_Factor"] = df_merged["Emissions_kg"] / df_merged["Tonnes"]
df_merged = df_merged[df_merged["Emission_Factor"].notna()]

# === STEP 5: ESTIMATE EMISSIONS FROM EXPORT ===
df_merged["Estimated_Emissions_kg"] = df_merged["Tonnes"] * df_merged["Emission_Factor"]

# === STEP 6: LOAD + MERGE POPULATION DATA ===
df_pop = df_pop[df_pop["Element"] == "Total Population - Both sexes"]
df_pop = df_pop[["Area", "Year", "Value"]]
df_pop.columns = ["Country", "Year", "Population_1000s"]
df_pop["Population"] = df_pop["Population_1000s"] * 1000

df_final = pd.merge(df_merged, df_pop, on=["Country", "Year"], how="left")
df_final = df_final[df_final["Population"].notna()]

# === STEP 7: CALCULATE PER CAPITA METRICS ===
df_final["Tonnes_per_capita"] = df_final["Tonnes"] / df_final["Population"]
df_final["Emissions_per_capita"] = df_final["Estimated_Emissions_kg"] / df_final["Population"]

# === STEP 8: AGGREGATE FOR VISUALISATION ===
def group_for_plot(value_column):
    return df_final.groupby(["Country", "Item"])[value_column].sum().unstack().fillna(0)

volume_df = group_for_plot("Tonnes")
emissions_df = group_for_plot("Estimated_Emissions_kg")
volume_pc_df = group_for_plot("Tonnes_per_capita")
emissions_pc_df = group_for_plot("Emissions_per_capita")

# === STEP 9: BUILD INTERACTIVE VISUALISATION ===
fig = go.Figure()

# Add default view (volume)
for item in volume_df.columns:
    fig.add_trace(go.Bar(name=item, x=volume_df.index, y=volume_df[item], visible=True))

# Add other 3 datasets as hidden
for df_data in [emissions_df, volume_pc_df, emissions_pc_df]:
    for item in df_data.columns:
        fig.add_trace(go.Bar(name=item, x=df_data.index, y=df_data[item], visible=False))

# Dropdown logic
n = len(volume_df.columns)
dropdown_buttons = [
    dict(label="Export Volume (tonnes)",
         method="update",
         args=[{"visible": [True]*n + [False]*(3*n)},
               {"title": "Export Volume by Country (tonnes)",
                "yaxis": {"title": "Tonnes"}}]),
    dict(label="Estimated CO₂ Emissions (kg)",
         method="update",
         args=[{"visible": [False]*n + [True]*n + [False]*(2*n)},
               {"title": "Estimated CO₂ Emissions by Country",
                "yaxis": {"title": "kg CO₂"}}]),
    dict(label="Export Volume per Capita (tonnes/person)",
     method="update",
     args=[{"visible": [False]*(2*n) + [True]*n + [False]*n},
           {"title": "Export Volume per Capita by Country",
            "yaxis": {"title": "Tonnes per person"}}]),
    dict(label="CO₂ Emissions per Capita (kg/person)",
        method="update",
        args=[{"visible": [False]*(3*n) + [True]*n},
           {"title": "Export-Linked CO₂ Emissions per Capita by Country",
            "yaxis": {"title": "kg CO₂ per person"}}])

]

# Final layout
fig.update_layout(
    updatemenus=[dict(active=0, buttons=dropdown_buttons, x=1.05)],
    barmode="stack",
    title="Export Volume by Country (tonnes)",
    xaxis_title="Country",
    yaxis_title="Tonnes",
    legend_title="Product",
    height=650,
    margin=dict(l=50, r=50, t=80, b=50)
)

fig.show()



SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' (1557279461.py, line 82)